    # What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary.

Given that our stakeholders are seeking to understand what factors drive the price of a used car then their goal is to improve their sales and meet market demand. They have provided as a dataset of used car sales and have asked us to determine which factors make a car more or less expensive. A recent [report](https://www.statista.com/statistics/183713/value-of-us-passenger-cas-sales-and-leases-since-1990/) shows that used car sales doubles the amount of new car sales. The used car market is lucrative with high demand from consumers. Our stakeholder wants to stay ahead of the curve and understanding their customer's preferences can help lead to more sales. Used car dealerships also have to acquire cars for inventory from other entities. If our stakeholder can reasonably determine the cost of a used vehicle they can better understand their margins and when to purchase a vehicle for inventory. It is also important to understand what customers value in used cars. Knowing what drives a consumer to purchase a used vehicle can help our stakeholders avoid costly mistakes when acquiring inventory.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
import os.path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import (
    mean_squared_error,
    root_mean_squared_error,
    mean_absolute_error,
)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
)
from sklearn.exceptions import ConvergenceWarning
from warnings import filterwarnings

filterwarnings("ignore")
filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
# Load data
vehicles_df = pd.read_csv("data/vehicles.csv")

In [ ]:
# First we should get the basic facts of the data (describe, info, null checks, duplicate checks)
vehicles_df.info()

The id and VIN column are not important for analysis so we will definitely remove them.

In [ ]:
vehicles_df.describe()

In [ ]:
vehicles_df.describe(include="object")

In [ ]:
print(
    f"Row count: {vehicles_df.shape[0]}, Duplicate count: {vehicles_df.shape[0] - vehicles_df.drop_duplicates().shape[0]}"
)

In [ ]:
vehicles_df.isna().mean().round(2)

In [ ]:
vehicles_df.sample(5)

Luckily, we do not have duplicates. However, we are missing values, specifically the condition and cylinders attributes are missing in over 40% of rows. Color is missing in 31% of rows. Size is missing in 72% of rows. Drive is missing in 31% of rows. I know that these attributes are important in determining the price of a car from prior analysis. The amount of missing data is concerning and the best course of action may be to impute them. That may mislead our model and we should be wary of that. However, dropping over 40% of the data isn't something we should do. One way to impute this data would be to find a finding an identical year and model then we would select the most seen value for that respective column; we can leverage scipy's `SimpleImputer`. This can help us deduce the missing attributes.

I will create an imputed dataset and run my analysis on both of them.

The imputed vehicle dataframe is looking a lot better.
<table>
<tr>
<th>column</th>
<th>old missing value percent</th>
<th>new missing value percent</th>
</tr>
<tr>
<td>condition</td>
<td>0.41</td>
<td>0.09</td>
</tr>
<tr>
<td>condition</td>
<td>0.42</td>
<td>0.14</td>
</tr>
<td>drive</td>
<td>0.31</td>
<td>0.10</td>
</tr>
</tr>
<td>size</td>
<td>0.72</td>
<td>0.30</td>
</tr>
</tr>
<td>type</td>
<td>0.22</td>
<td>0.05</td>
</tr>
</tr>
<td>paint_color</td>
<td>0.31</td>
<td>0.08</td>
</tr>
</table?


Let's start by checking a histogram of sales prices

In [ ]:
fig = px.box(
    vehicles_df,
    x="price",
    title="Price attribute has unrealistic values",
    labels={"price": "Price", "count": "Count"},
)
fig.show()
fig.write_image("images/box_amount_with_outliers.png")

In [ ]:
fig = px.box(
    vehicles_df,
    x="year",
    labels={"year": "Year", "count": "Count"},
    title="Most cars sold are 2008 or older models",
)
fig.show()
fig.write_image("images/year_box_with_outliers.png")

In [ ]:
# predefine to avoid extra computation
groupby_model_year = vehicles_df.groupby(["model", "year"])
fields_to_impute = ["condition", "cylinders", "drive", "size", "type", "paint_color"]


# This somewhat mimics SimpleImputer. If we wanted to get fancy we would use KNN and leverage sales price in imputation
def impute_field_by_model_year(row, field_to_update):
    # ensure fields are not na and some non na value exists
    try:
        # take the most common occurrences for model and year
        group = groupby_model_year.get_group((row["model"], row["year"]))
        non_na_values = group[field_to_update].dropna()
        if not non_na_values.empty:
            return non_na_values.mode().iloc[0]
    except KeyError:
        return np.nan
    return np.nan


# Impute once since it can be expensive
def load_imputed_data():
    if not os.path.exists("data/imputed_vehicles.csv"):
        # copy vehicles df and impute columns missing values
        imputed_vehicle_df = vehicles_df.copy(deep=True)
        for field in fields_to_impute:
            na_filter = imputed_vehicle_df[field].isna()
            imputed_vehicle_df.loc[na_filter, field] = imputed_vehicle_df[
                na_filter
            ].apply(lambda row: impute_field_by_model_year(row, field), axis=1)

        imputed_vehicle_df.to_csv("data/imputed_vehicles.csv")
    else:
        imputed_vehicle_df = pd.read_csv("data/imputed_vehicles.csv")
    return imputed_vehicle_df


imputed_vehicle_df = load_imputed_data()

In [ ]:
imputed_vehicle_df.isna().mean().round(2)

The above is not realistic. We are seeing vehicles selling for billions. I will remove outliers using iqr. We also see very old cars (likely vintage) in the outlier group for year. I will remove those as well.

In [ ]:
q1 = vehicles_df["price"].quantile(0.25)
q3 = vehicles_df["price"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

vehicles_df_no_outlier = vehicles_df.query(
    f"price >= {lower_bound} and price <= {upper_bound} and year > 1995"
)
imputed_vehicle_df = imputed_vehicle_df.query(
    f"price >= {lower_bound} and price <= {upper_bound} and year > 1995"
)

In [ ]:
# Now we can drop na
vehicles_df_no_outlier.dropna(inplace=True)
imputed_vehicle_df.dropna(inplace=True)
print(vehicles_df_no_outlier.shape[0])
print(imputed_vehicle_df.shape[0])

After removing outliers & dropping rows with NAs the non imputed dataset contains 33,728 rows and the imputed set contains 148,360. This is out of the original 426,880 rows.

In [ ]:
fig = px.box(
    vehicles_df_no_outlier,
    x="price",
    title="Majority of vehicles fall between &#36;6,500 - &#36;21,990",
    labels={"price": "Price", "count": "Count"},
)
fig.show()
fig.write_image("images/box_amount.png")

In [ ]:
fig = px.histogram(
    vehicles_df_no_outlier,
    x="price",
    nbins=10,
    labels={"price": "Price", "count": "Count"},
    title="A majority of vehicles are bought at a lower price",
)
fig.show()
fig.write_image("images/price_histogram.png")

In [ ]:
fig = px.histogram(
    vehicles_df_no_outlier,
    x="year",
    nbins=10,
    labels={"year": "Year", "count": "Count"},
    title="Most cars sold are newer",
)
fig.show()
fig.write_image("images/year_histogram.png")

In [ ]:
# Univariate analysis with the imputed columns. Compare between non imputed vs imputed
fields_to_impute = ["condition", "cylinders", "drive", "size", "type", "paint_color"]

fig = make_subplots(
    rows=len(fields_to_impute),
    cols=2,
    subplot_titles=(
        "Condition: Good and excellent were sold the most",
        "Condition: Imputation increased the amount of excellent vehicles, should be cautious",
        "Cylinders: 6, 4 & 8 cylinders sold the most",
        "Cylinders: Imputation follows the ratio",
        "Drive: 4 wheel drive is the most popular",
        "Drive: Imputation follows the ratio",
        "Color: White, black and silver make the majority of the sales",
        "Color: Imputation follows the ratio",
        "Size: Full size is the most popular",
        "Size: Imputation follows the ratio",
        "Type: Sedan, SUVs and pickups make the majority of the sales",
        "Type: Imputation shuffled coupes, hatchbacks, convertibles and mini-vans",
    ),
)

for i, field in enumerate(sorted(fields_to_impute)):
    no_outlier_counts = vehicles_df_no_outlier[field].value_counts()
    bar_vehicle_fig = go.Bar(x=no_outlier_counts.index, y=no_outlier_counts)
    fig.add_trace(bar_vehicle_fig, row=i + 1, col=1)

    imputed_counts = imputed_vehicle_df[field].value_counts()
    bar_imputed_fig = go.Bar(x=imputed_counts.index, y=imputed_counts)
    fig.add_trace(bar_imputed_fig, row=i + 1, col=2)

fig.update_layout(height=1400, width=1400, showlegend=False)
fig.show()
fig.write_image("images/imputed_fields_comparison.png")

Imputation had some effect on our data. In particular, condition and type shuffled in terms of sales count. This is important because condition is so instrumental in determining sales price and the imputation may affect our model. We saw that type saw impact in the types with lower sales and that may not affect our model. We will create models for both datasets and use metrics to determine if our imputation helped or hurt our cause.

In [ ]:
# Univariate analysis with the non-imputed columns
non_altered_columns = list(
    filter(
        lambda field: field not in fields_to_impute and field != "VIN",
        list(vehicles_df_no_outlier.select_dtypes(include="object").columns),
    )
)
fig = make_subplots(
    rows=len(non_altered_columns),
    cols=1,
    subplot_titles=(
        "Fuel: Gas is the most popular type",
        "Manufacturer: Ford, Chevy and Toyota are the most popular manufacturer",
        "Model: Preference is scattered. F150 tops. We see normalization of model names was not done (see silverado)",
        "Region: Should represent this differently",
        "State: Most populous states have most sales as expected. Should represent this differently",
        "Title status: Clean title is preferred",
        "Transmission: Automatic is the most popular",
    ),
)

for i, field in enumerate(sorted(non_altered_columns)):
    no_outlier_counts = vehicles_df_no_outlier[field].value_counts()
    # if a value represent less than 1% of the rows we will put it in the other column
    total_counts = no_outlier_counts.sum()
    percent_threshold = 0.01  # 1%
    other_mask = (no_outlier_counts / total_counts) < percent_threshold
    other_sum = no_outlier_counts[other_mask].sum()
    no_outlier_counts_filtered = no_outlier_counts[~other_mask]
    if "other" in list(no_outlier_counts_filtered.index):
        no_outlier_counts_filtered["other"] = (
            other_sum + no_outlier_counts_filtered["other"]
        )
    else:
        no_outlier_counts_filtered["other"] = other_sum
    bar_vehicle_fig = go.Bar(
        x=no_outlier_counts_filtered.index, y=no_outlier_counts_filtered
    )
    fig.add_trace(bar_vehicle_fig, row=i + 1, col=1)

fig.update_layout(height=1400, width=1400, showlegend=False)
fig.show()
fig.write_image("images/univariate_non_imputed_fields_bar.png")

The above analysis shows that the region, model and state should be represented differently.


In [ ]:
fig = px.treemap(
    vehicles_df_no_outlier,
    path=[px.Constant("USA"), "state", "region"],
    values="price",
    color="price",
    hover_data=["region"],
    color_continuous_scale="Magma",
    color_continuous_midpoint=np.average(vehicles_df_no_outlier["price"]),
    title="California has highest volume of sales and North Carolina has highest average",
)
fig.update_layout(height=1000, width=2000, showlegend=False)
fig.show()
fig.write_image("images/treemap_no_outlier.png")

In [ ]:
fig = px.treemap(
    imputed_vehicle_df,
    path=[px.Constant("USA"), "state", "region"],
    values="price",
    color="price",
    hover_data=["region"],
    color_continuous_scale="Magma",
    color_continuous_midpoint=np.average(imputed_vehicle_df["price"]),
    title="California retains highest volume of sales but now Washington has highest average",
)
fig.update_layout(height=1000, width=2000, showlegend=False)

fig.show()
fig.write_image("images/treemap_imputated.png")

With the treemaps we see that the most populated states have the highest volume. However, other states have a higher price average. This could be indicative of different needs in different regions.

In [ ]:
fig = px.treemap(
    vehicles_df_no_outlier,
    path=["manufacturer", "model"],
    values="price",
    color="price",
    hover_data=["region"],
    color_continuous_scale="Magma",
    color_continuous_midpoint=np.average(vehicles_df_no_outlier["price"]),
    title="Model does vary by price but the high cardinality may be bad for a model",
)
fig.update_layout(height=1000, width=2000, showlegend=False)

fig.show()
fig.write_image("images/treemap_model_no_outlier.png")

In [ ]:
avg_price_by_manufacturer = (
    vehicles_df_no_outlier.groupby("manufacturer")["price"]
    .mean()
    .sort_values()
    .reset_index()
)
fig = px.bar(
    avg_price_by_manufacturer,
    x="manufacturer",
    y="price",
    title="Ford is the most popular yet has a higher average price. Ferrari is missing data",
)
fig.update_layout(height=1000, width=1000)
fig.show()
fig.write_image("images/avg_price_by_manufacturer_no_outlier.png")

In [ ]:
avg_price_by_manufacturer = (
    imputed_vehicle_df.groupby("manufacturer")["price"]
    .mean()
    .sort_values()
    .reset_index()
)
fig = px.bar(
    avg_price_by_manufacturer,
    x="manufacturer",
    y="price",
    title="With imputation Telsa takes the highest price",
)
fig.show()
fig.write_image("images/avg_price_by_manufacturer_imputed.png")

I have seen enough to conclude that our imputed data set is good enough to work with moving forward. I will still build models for each to determine the which is best.

In [ ]:
# Let's do an average price by feature bar char
category_columns_to_ignore = [
    "VIN",
    "region",
    "model",
    "manufacturer",
    "state",
    "transmission",
]
category_columns_graph = list(
    filter(
        lambda field: field not in category_columns_to_ignore,
        list(vehicles_df_no_outlier.select_dtypes(include="object").columns),
    )
)
fig = make_subplots(
    rows=len(category_columns_graph),
    cols=1,
    subplot_titles=(
        "Condition: New has the highest average price, typically better condition equals higher average price",
        "Cylinders: More cylinders means more power means higher average price",
        "Drive: 4 wheel drive is more expensive on average",
        "Fuel: Diesel is the most expensive on average",
        "Color: Has an affect on average price",
        "Size: Full size vehicles are more expensive on average",
        "Title Status: Vehicles with liens are more expensive, likely because the lien needs to be paid off",
        "Type: Trucks, pickups and offroad vehicles are more expensive on average",
    ),
)

for i, field in enumerate(sorted(category_columns_graph)):
    no_outlier_counts = imputed_vehicle_df[field].value_counts()
    avg_price_by_field = (
        imputed_vehicle_df.groupby(field)["price"].mean().sort_values().reset_index()
    )
    bar_fig = go.Bar(x=avg_price_by_field[field], y=avg_price_by_field["price"])
    fig.add_trace(bar_fig, row=i + 1, col=1)

fig.update_layout(height=1400, width=1400, showlegend=False)
fig.show()
fig.write_image("images/average_price_by_field.png")

In [ ]:
# Correlation plots
corr_matrix = (
    imputed_vehicle_df[["price", "year", "odometer"]].corr(numeric_only=True).round(2)
)
fig = px.imshow(
    corr_matrix,
    title="Newer cars are priced higher, mileage is negatively correlated with year and price",
    color_continuous_scale="RdBu_r",
    aspect="auto",
)
fig.update_layout(height=1000, width=1000, showlegend=False)
fig.show()
fig.write_image("images/correlation.png")

This correlation gave me the idea to add the feature below. Average miles per year.

In [ ]:
current_year = 2025
vehicles_df_no_outlier["age"] = current_year - vehicles_df_no_outlier["year"]
imputed_vehicle_df["age"] = current_year - imputed_vehicle_df["year"]
vehicles_df_no_outlier["avg_miles_per_year"] = (
    vehicles_df_no_outlier["odometer"] / vehicles_df_no_outlier["age"]
)
imputed_vehicle_df["avg_miles_per_year"] = (
    vehicles_df_no_outlier["odometer"] / vehicles_df_no_outlier["age"]
)

In [ ]:
fig = px.histogram(
    imputed_vehicle_df,
    x="avg_miles_per_year",
    labels={"avg_miles_per_year": "Avg miles per year", "count": "Count"},
    title="Average miles per year follows a normal distribution but we have outliers",
)
fig.show()
fig.write_image("images/avg_miles_per_year_histogram.png")

In [ ]:
fig = px.box(
    imputed_vehicle_df,
    x="avg_miles_per_year",
    labels={"avg_miles_per_year": "Avg miles per year"},
    title="Most vehicles are adding 5k-10k miles a year",
)
fig.show()
fig.write_image("images/avg_miles_per_year_box.png")

In [ ]:
fig = px.bar(
    imputed_vehicle_df.groupby("age")["price"].mean().sort_values().reset_index(),
    y="price",
    x="age",
    title="Price decreases as age increases",
    labels={"price": "Price", "age": "Age"},
)
fig.show()
fig.write_image("images/average_price_by_age.png")

In [ ]:
# Correlation plots
corr_matrix = (
    imputed_vehicle_df[["price", "year", "odometer", "avg_miles_per_year", "age"]]
    .corr(numeric_only=True)
    .round(2)
)
fig = px.imshow(
    corr_matrix,
    title="Avg miles per year shows no correlation with price, indicates non linear relationship",
    color_continuous_scale="RdBu_r",
    aspect="auto",
)
fig.update_layout(height=1000, width=1000)
fig.show()
fig.write_image("images/correlation_new_features.png")

In [ ]:
vehicles_df_numeric = imputed_vehicle_df.select_dtypes("number").drop(
    ["price", "Unnamed: 0", "id"], axis=1
)

pca = PCA(n_components=4).fit(vehicles_df_numeric.dropna())

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

fig = px.line(
    explained_variance,
    labels={"index": "Principal Component", "value": "Explained Variance Ratio"},
    title="With only two numerical features tough to use PCA",
    markers=True,
)
fig.update_layout(showlegend=False)
fig.show()
fig.write_image("images/pca.png")

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`.

In the data understanding section, I imputed data, removed outliers and added new features as part. I will repeat the process.

In [ ]:
# Reload dataset
vehicles_df = pd.read_csv("data/vehicles.csv")

In [ ]:
# Drop columns (id, vin, region)
vehicles_df = vehicles_df.drop(["id", "VIN", "region"], axis=1)

In [ ]:
# Remove outliers
q1 = vehicles_df["price"].quantile(0.25)
q3 = vehicles_df["price"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
vehicles_df = vehicles_df.query(
    f"price >= {lower_bound} and price <= {upper_bound} and year > 1995"
)

In [ ]:
# impute data
# predefine to avoid extra computation
groupby_model_year = vehicles_df.groupby(["model", "year"])
fields_to_impute = ["condition", "cylinders", "drive", "size", "type", "paint_color"]


# Impute once since it can be expensive
def load_imputed_data_data_prep():
    for field in fields_to_impute:
        na_filter = vehicles_df[field].isna()
        vehicles_df.loc[na_filter, field] = vehicles_df[na_filter].apply(
            lambda row: impute_field_by_model_year(row, field), axis=1
        )

    return vehicles_df


vehicles_df = load_imputed_data_data_prep()

In [ ]:
# Check impute change
vehicles_df.isna().mean().round(2)

In [ ]:
# Add features
current_year = 2025
vehicles_df["age"] = current_year - vehicles_df["year"]
vehicles_df["avg_miles_per_year"] = vehicles_df["odometer"] / vehicles_df["age"]

In [ ]:
# Drop duplicates and rows with NA
vehicles_df = vehicles_df.drop_duplicates()
vehicles_df = vehicles_df.dropna()

In [ ]:
vehicles_df.shape

In [ ]:
vehicles_df["title_status"].unique()

In [ ]:
vehicles_df["condition"].unique()

In [ ]:
# define our columns.
numerical_columns = ["age", "odometer", "avg_miles_per_year"]
category_columns = [
    "manufacturer",
    "cylinders",
    "fuel",
    "transmission",
    "drive",
    "size",
    "type",
    "paint_color",
    "state",
]
title_status_columns = ["title_status"]
condition_columns = ["condition"]
title_statuses = ["clean", "lien", "rebuilt", "salvage", "missing", "parts only"]
condition_values = ["new", "like new", "excellent", "good", "fair", "salvage"]

# added after evaluation, remove paint as a feature
category_columns_no_paint = [
    "manufacturer",
    "cylinders",
    "fuel",
    "transmission",
    "drive",
    "size",
    "type",
    "state",
]

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("poly", PolynomialFeatures(include_bias=False), numerical_columns),
        (
            "ordinal_title",
            OrdinalEncoder(categories=[title_statuses]),
            title_status_columns,
        ),
        (
            "ordinal_condition",
            OrdinalEncoder(categories=[condition_values]),
            condition_columns,
        ),
        ("onehot", OneHotEncoder(handle_unknown="ignore"), category_columns),
    ]
)

# added after evaluation, remove paint as a feature
preprocessor_no_paint = ColumnTransformer(
    transformers=[
        ("poly", PolynomialFeatures(include_bias=False), numerical_columns),
        (
            "ordinal_title",
            OrdinalEncoder(categories=[title_statuses]),
            title_status_columns,
        ),
        (
            "ordinal_condition",
            OrdinalEncoder(categories=[condition_values]),
            condition_columns,
        ),
        ("onehot", OneHotEncoder(handle_unknown="ignore"), category_columns_no_paint),
    ]
)

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
def eval_model(pipe, model_name, X_train, X_test, y_train, y_test, best_params):
    train_mae = mean_absolute_error(pipe.predict(X_train), y_train)
    train_mse = mean_squared_error(pipe.predict(X_train), y_train)
    train_rmse = root_mean_squared_error(pipe.predict(X_train), y_train)

    test_mae = mean_absolute_error(pipe.predict(X_test), y_test)
    test_mse = mean_squared_error(pipe.predict(X_test), y_test)
    test_rmse = root_mean_squared_error(pipe.predict(X_test), y_test)

    return {
        "model_name": model_name,
        "train_mae": train_mae,
        "test_mae": test_mae,
        "train_mse": train_mse,
        "test_mse": test_mse,
        "train_rmse": train_rmse,
        "test_rmse": test_rmse,
        "coef": (
            pipe.named_steps[model_name].coef_
            if hasattr(pipe.named_steps[model_name], "coef_")
            else None
        ),
        "best_params": best_params,
    }

In [ ]:
scores = []

In [ ]:
# Split data
X = vehicles_df.drop("price", axis=1)
y = vehicles_df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# LinearRegression

linreg_pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler(with_mean=False)),
        ("LinearRegression", LinearRegression()),
    ]
)

param_grid = {
    "preprocessor__poly__degree": [2, 3, 4, 5, 6],
}

grid_linreg = GridSearchCV(
    estimator=linreg_pipe, param_grid=param_grid, cv=5, n_jobs=-1
)
grid_linreg.fit(X_train, y_train)
print(grid_linreg.best_params_)

In [ ]:
# Added after model evaluation. Remove paint as a feature
linreg_pipe_no_paint = Pipeline(
    [
        ("preprocessor", preprocessor_no_paint),
        ("scaler", StandardScaler(with_mean=False)),
        ("LinearRegressionNoPaint", LinearRegression()),
    ]
)

param_grid = {
    "preprocessor__poly__degree": [2, 3, 4, 5, 6],
}

grid_linreg_no_paint = GridSearchCV(
    estimator=linreg_pipe_no_paint, param_grid=param_grid, cv=5, n_jobs=-1
)
grid_linreg_no_paint.fit(X_train, y_train)
print(grid_linreg_no_paint.best_params_)

In [ ]:
linreg_evaluation = eval_model(
    grid_linreg.best_estimator_,
    "LinearRegression",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_linreg.best_params_,
)
scores.append(linreg_evaluation)

# Added after model evaluation. Remove paint as a feature
linreg_evaluation_no_paint = eval_model(
    grid_linreg_no_paint.best_estimator_,
    "LinearRegressionNoPaint",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_linreg_no_paint.best_params_,
)
scores.append(linreg_evaluation_no_paint)

In [ ]:
# Ridge
ridge_pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler(with_mean=False)),
        ("Ridge", Ridge()),
    ]
)

param_grid = {
    "preprocessor__poly__degree": [2, 3, 4, 5, 6],
    "Ridge__alpha": [0.01, 0.1, 1, 10, 100],
}

grid_ridge = GridSearchCV(estimator=ridge_pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid_ridge.fit(X_train, y_train)
print(grid_ridge.best_params_)

In [ ]:
# Added after model evaluation. Remove paint as feature
ridge_pipe_no_paint = Pipeline(
    [
        ("preprocessor", preprocessor_no_paint),
        ("scaler", StandardScaler(with_mean=False)),
        ("RidgeNoPaint", Ridge()),
    ]
)

param_grid = {
    "preprocessor__poly__degree": [2, 3, 4, 5, 6],
    "RidgeNoPaint__alpha": [0.01, 0.1, 1, 10, 100],
}

grid_ridge_no_paint = GridSearchCV(
    estimator=ridge_pipe_no_paint, param_grid=param_grid, cv=5, n_jobs=-1
)
grid_ridge_no_paint.fit(X_train, y_train)
print(grid_ridge_no_paint.best_params_)

In [ ]:
ridge_evaluation = eval_model(
    grid_ridge.best_estimator_,
    "Ridge",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_ridge.best_params_,
)
scores.append(ridge_evaluation)

# Added after model evaluation. Remove paint as a feature
ridge_evaluation_no_paint = eval_model(
    grid_ridge_no_paint.best_estimator_,
    "RidgeNoPaint",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_ridge_no_paint.best_params_,
)
scores.append(ridge_evaluation_no_paint)

In [ ]:
# Lasso
# Was seeing ConvergenceWarning so ignoring it above, reduce degree for speed. Saw no advantage above 3
lasso_pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler(with_mean=False)),
        ("Lasso", Lasso(max_iter=15000, tol=1e-1)),
    ]
)


param_grid = {
    "preprocessor__poly__degree": [2, 3, 4],
    "Lasso__alpha": [1.0, 5, 10, 50, 100],
}

grid_lasso = GridSearchCV(estimator=lasso_pipe, param_grid=param_grid, cv=3, n_jobs=-1)
grid_lasso.fit(X_train, y_train)
print(grid_lasso.best_params_)

In [ ]:
# Added after model evaluation. Remove paint as a feature
lasso_pipe_no_paint = Pipeline(
    [
        ("preprocessor", preprocessor_no_paint),
        ("scaler", StandardScaler(with_mean=False)),
        ("LassoNoPaint", Lasso(max_iter=15000, tol=1e-1)),
    ]
)


param_grid = {
    "preprocessor__poly__degree": [2, 3, 4],
    "LassoNoPaint__alpha": [1.0, 5, 10, 50, 100],
}

grid_lasso_no_paint = GridSearchCV(
    estimator=lasso_pipe_no_paint, param_grid=param_grid, cv=3, n_jobs=-1
)
grid_lasso_no_paint.fit(X_train, y_train)
print(grid_lasso_no_paint.best_params_)

In [ ]:
lasso_evaluation = eval_model(
    grid_lasso.best_estimator_,
    "Lasso",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_lasso.best_params_,
)
scores.append(lasso_evaluation)

lasso_evaluation_no_paint = eval_model(
    grid_lasso_no_paint.best_estimator_,
    "LassoNoPaint",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_lasso_no_paint.best_params_,
)
scores.append(lasso_evaluation_no_paint)

In [ ]:
# RandomForest
random_forest_pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler(with_mean=False)),
        ("RandomForest", RandomForestRegressor()),
    ]
)
param_grid = {
    "preprocessor__poly__degree": [2, 3, 4],
    "RandomForest__max_depth": [4, 5],
    "RandomForest__n_estimators": [100, 200],
}

grid_random_forest = GridSearchCV(
    estimator=random_forest_pipe, param_grid=param_grid, cv=3, n_jobs=-1
)
grid_random_forest.fit(X_train, y_train)
print(grid_random_forest.best_params_)

In [ ]:
# Added after model evaluation. Remove paint as a feature
random_forest_pipe_no_paint = Pipeline(
    [
        ("preprocessor", preprocessor_no_paint),
        ("scaler", StandardScaler(with_mean=False)),
        ("RandomForestNoPaint", RandomForestRegressor()),
    ]
)
param_grid = {
    "preprocessor__poly__degree": [2, 3, 4],
    "RandomForestNoPaint__max_depth": [4, 5],
    "RandomForestNoPaint__n_estimators": [100, 200],
}

grid_random_forest_no_paint = GridSearchCV(
    estimator=random_forest_pipe_no_paint, param_grid=param_grid, cv=3, n_jobs=-1
)
grid_random_forest_no_paint.fit(X_train, y_train)
print(grid_random_forest_no_paint.best_params_)

In [ ]:
random_forest_evaluation = eval_model(
    grid_random_forest.best_estimator_,
    "RandomForest",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_random_forest.best_params_,
)
scores.append(random_forest_evaluation)

random_forest_evaluation_no_paint = eval_model(
    grid_random_forest_no_paint.best_estimator_,
    "RandomForestNoPaint",
    X_train,
    X_test,
    y_train,
    y_test,
    grid_random_forest_no_paint.best_params_,
)
scores.append(random_forest_evaluation_no_paint)

In [ ]:
scores_df = pd.DataFrame(scores)
scores_df.to_csv("data/scores_df.csv")

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [ ]:
def load_scores_data():
    if os.path.exists("data/scores_df.csv"):
        return pd.read_csv("data/scores_df.csv")
    print("Scores csv doesn't exist, train and score models first")
    return None

In [ ]:
scores_df = load_scores_data()

In [ ]:
fig = go.Figure()

train_bar = go.Bar(
    x=scores_df["model_name"], y=scores_df["train_mae"], name="Train MAE"
)
fig.add_trace(train_bar)
test_bar = go.Bar(x=scores_df["model_name"], y=scores_df["test_mae"], name="Test MAE")
fig.add_trace(test_bar)

fig.update_layout(
    title_text="Train vs Test MAE Comparison Small gap shows good generalization. Ridge most performant",
    xaxis_title="Model",
    yaxis_title="Mean Absolute Error (MAE)",
    barmode="group",
    height=600,
    width=1000,
    showlegend=True,
    bargap=0.5,
)
fig.show()
fig.write_image("images/mae_comparison.png")

In [ ]:
fig = go.Figure()

train_bar = go.Bar(
    x=scores_df["model_name"], y=scores_df["train_mse"], name="Train MSE"
)
fig.add_trace(train_bar)
test_bar = go.Bar(x=scores_df["model_name"], y=scores_df["test_mse"], name="Test MSE")
fig.add_trace(test_bar)

fig.update_layout(
    title_text="Train vs Test MSE Comparison Small gap shows good generalization. Ridge most performant",
    xaxis_title="Model",
    yaxis_title="Mean Squared Error (MSE)",
    barmode="group",
    height=600,
    width=1000,
    showlegend=True,
    bargap=0.5,
)
fig.show()
fig.write_image("images/mse_comparison.png")

In [ ]:
fig = go.Figure()

train_bar = go.Bar(
    x=scores_df["model_name"], y=scores_df["train_rmse"], name="Train RMSE"
)
fig.add_trace(train_bar)
test_bar = go.Bar(x=scores_df["model_name"], y=scores_df["test_rmse"], name="Test RMSE")
fig.add_trace(test_bar)

fig.update_layout(
    title_text="Train vs Test RMSE Comparison Small gap shows good generalization. Ridge most performant",
    xaxis_title="Model",
    yaxis_title="Root Mean Squared Error (RMSE)",
    barmode="group",
    height=600,
    width=1000,
    showlegend=True,
    bargap=0.5,
)
fig.show()
fig.write_image("images/rmse_comparison.png")

In [ ]:
def get_feature_names(grid, is_paint=True):
    preprocessor = grid.best_estimator_.named_steps["preprocessor"]
    poly_features = preprocessor.named_transformers_["poly"].get_feature_names_out(numerical_columns)
    ordinal_title_features = preprocessor.named_transformers_["ordinal_title"].get_feature_names_out(title_status_columns)
    ordinal_condition_features = preprocessor.named_transformers_["ordinal_condition"].get_feature_names_out(condition_columns)
    if is_paint:
        onehot_features = preprocessor.named_transformers_["onehot"].get_feature_names_out(category_columns)
    else:
        onehot_features = preprocessor.named_transformers_["onehot"].get_feature_names_out(category_columns_no_paint)
    return np.concatenate(
        [
            poly_features,
            ordinal_title_features,
            ordinal_condition_features,
            onehot_features,
        ]
    )

In [ ]:
# Get feature importance for each model

linear_importance = {
    "Feature": get_feature_names(grid_linreg),
    "Importance - Coef": np.abs(
        grid_linreg.best_estimator_.named_steps["LinearRegression"].coef_
    ),
}
linear_importance_df = pd.DataFrame(linear_importance)

ridge_importance = {
    "Feature": get_feature_names(grid_ridge),
    "Importance - Coef": np.abs(grid_ridge.best_estimator_.named_steps["Ridge"].coef_),
}
ridge_importance_df = pd.DataFrame(ridge_importance)

lasso_importance = {
    "Feature": get_feature_names(grid_lasso),
    "Importance - Coef": np.abs(grid_lasso.best_estimator_.named_steps["Lasso"].coef_),
}
lasso_importance_df = pd.DataFrame(lasso_importance)

random_forest_importance = {
    "Feature": get_feature_names(grid_random_forest),
    "Importance - Coef": np.abs(
        grid_random_forest.best_estimator_.named_steps[
            "RandomForest"
        ].feature_importances_
    ),
}
random_forest_importance_df = pd.DataFrame(random_forest_importance)

# Now adding estimators without paint

linear_importance_no_paint = {
    "Feature": get_feature_names(grid_linreg_no_paint, is_paint=False),
    "Importance - Coef": np.abs(
        grid_linreg_no_paint.best_estimator_.named_steps[
            "LinearRegressionNoPaint"
        ].coef_
    ),
}
linear_importance_no_paint_df = pd.DataFrame(linear_importance_no_paint)

ridge_importance_no_paint = {
    "Feature": get_feature_names(grid_ridge_no_paint, is_paint=False),
    "Importance - Coef": np.abs(
        grid_ridge_no_paint.best_estimator_.named_steps["RidgeNoPaint"].coef_
    ),
}
ridge_importance_no_paint_df = pd.DataFrame(ridge_importance_no_paint)

lasso_importance_no_paint = {
    "Feature": get_feature_names(grid_lasso_no_paint, is_paint=False),
    "Importance - Coef": np.abs(
        grid_lasso_no_paint.best_estimator_.named_steps["LassoNoPaint"].coef_
    ),
}
lasso_importance_no_paint_df = pd.DataFrame(lasso_importance_no_paint)

random_forest_importance_no_paint = {
    "Feature": get_feature_names(grid_random_forest_no_paint, is_paint=False),
    "Importance - Coef": np.abs(
        grid_random_forest_no_paint.best_estimator_.named_steps[
            "RandomForestNoPaint"
        ].feature_importances_
    ),
}
random_forest_importance_no_paint_df = pd.DataFrame(random_forest_importance_no_paint)

In [ ]:
linear_importance_df = linear_importance_df.sort_values("Importance - Coef", ascending=False)
fig = px.bar(
    linear_importance_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features LinearRegression",
)
fig.show()
fig.write_image("images/linear_regression_top_features.png")

In [ ]:
linear_importance_no_paint_df = linear_importance_no_paint_df.sort_values(
    "Importance - Coef", ascending=False
)
fig = px.bar(
    linear_importance_no_paint_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features No Paint LinearRegression",
)
fig.show()
fig.write_image("images/linear_regression_no_paint_top_features.png")

In [ ]:
ridge_importance_df = ridge_importance_df.sort_values("Importance - Coef", ascending=False)
fig = px.bar(
    ridge_importance_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features Ridge",
)
fig.show()
fig.write_image("images/ridge_top_features.png")

In [ ]:
ridge_importance_no_paint_df = ridge_importance_no_paint_df.sort_values(
    "Importance - Coef", ascending=False
)
fig = px.bar(
    ridge_importance_no_paint_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features No Paint Ridge",
)
fig.show()
fig.write_image("images/ridge_no_paint_top_features.png")

In [ ]:
lasso_importance_df = lasso_importance_df[
    lasso_importance_df["Importance - Coef"] > 0
].sort_values("Importance - Coef", ascending=False)
fig = px.bar(
    lasso_importance_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features Lasso",
)
fig.show()
fig.write_image("images/lasso_top_features.png")

In [ ]:
lasso_importance_no_paint_df = lasso_importance_no_paint_df[
    lasso_importance_no_paint_df["Importance - Coef"] > 0
].sort_values("Importance - Coef", ascending=False)
fig = px.bar(
    lasso_importance_no_paint_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features Lasso",
)
fig.show()
fig.write_image("images/lasso_no_paint_top_features.png")

In [ ]:
random_forest_importance_df = random_forest_importance_df[
    random_forest_importance_df["Importance - Coef"] > 0
].sort_values("Importance - Coef", ascending=False)
fig = px.bar(
    random_forest_importance_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features RandomForest",
)
fig.show()
fig.write_image("images/random_forest_top_features.png")

In [ ]:
random_forest_importance_no_paint_df = random_forest_importance_no_paint_df[
    random_forest_importance_no_paint_df["Importance - Coef"] > 0
].sort_values("Importance - Coef", ascending=False)
fig = px.bar(
    random_forest_importance_no_paint_df[:10],
    x="Importance - Coef",
    y="Feature",
    orientation="h",
    title="Top Ten Features No Paint RandomForest",
)
fig.show()
fig.write_image("images/random_forest_no_paint_top_features.png")

I am going to go back and make models without paint in the feature set.

After testing, removing paint made our models worse! This was not what I thought would happen.

<table>
  <thead>
    <tr>
      <th>model_name</th>
      <th>train_mae</th>
      <th>test_mae</th>
      <th>train_mse</th>
      <th>test_mse</th>
      <th>train_rmse</th>
      <th>test_rmse</th>
      <th>best_params</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>LinearRegression</td>
      <td>5283.77</td>
      <td>5294.00</td>
      <td>64031541.68</td>
      <td>64194335.93</td>
      <td>8001.97</td>
      <td>8012.14</td>
      <td>{'preprocessor__poly__degree': 2}</td>
    </tr>
    <tr>
      <td>LinearRegressionNoPaint</td>
      <td>5285.40</td>
      <td>5296.52</td>
      <td>64098638.01</td>
      <td>64269098.87</td>
      <td>8006.16</td>
      <td>8016.80</td>
      <td>{'preprocessor__poly__degree': 2}</td>
    </tr>
    <tr>
      <td>Ridge</td>
      <td>5225.62</td>
      <td>5229.38</td>
      <td>63065606.60</td>
      <td>63064394.53</td>
      <td>7941.39</td>
      <td>7941.31</td>
      <td>{'Ridge__alpha': 0.01, 'preprocessor__poly__degree': 6}</td>
    </tr>
    <tr>
      <td>RidgeNoPaint</td>
      <td>5226.53</td>
      <td>5231.90</td>
      <td>63124370.98</td>
      <td>63128494.09</td>
      <td>7945.08</td>
      <td>7945.34</td>
      <td>{'RidgeNoPaint__alpha': 0.01, 'preprocessor__poly__degree': 6}</td>
    </tr>
    <tr>
      <td>Lasso</td>
      <td>5257.35</td>
      <td>5265.29</td>
      <td>63636373.78</td>
      <td>63650075.25</td>
      <td>7977.24</td>
      <td>7978.10</td>
      <td>{'Lasso__alpha': 1.0, 'preprocessor__poly__degree': 3}</td>
    </tr>
    <tr>
      <td>LassoNoPaint</td>
      <td>5258.66</td>
      <td>5267.62</td>
      <td>63700182.15</td>
      <td>63720917.01</td>
      <td>7981.24</td>
      <td>7982.54</td>
      <td>{'LassoNoPaint__alpha': 1.0, 'preprocessor__poly__degree': 3}</td>
    </tr>
    <tr>
      <td>RandomForest</td>
      <td>5418.38</td>
      <td>5457.99</td>
      <td>67308779.78</td>
      <td>67985841.22</td>
      <td>8204.19</td>
      <td>8245.35</td>
      <td>{'RandomForest__max_depth': 5, 'RandomForest__n_estimators': 100, 'preprocessor__poly__degree': 4}</td>
    </tr>
    <tr>
      <td>RandomForestNoPaint</td>
      <td>5417.84</td>
      <td>5458.16</td>
      <td>67338583.29</td>
      <td>68029615.54</td>
      <td>8206.01</td>
      <td>8248.01</td>
      <td>{'RandomForestNoPaint__max_depth': 5, 'RandomForestNoPaint__n_estimators': 200, 'preprocessor__poly__degree': 4}</td>
    </tr>
  </tbody>
</table>

<p>
After a thorough analysis of four different model types, the Ridge model stands out as the best. The Ridge model achieved a Mean Absolute Error of 5,229.38, a Mean Squared Error of 63,064,394.53 and a Root Square Error of 7,941.31 on the test set. The best parameters identified through cross-validation were an alpha of 0.01 and a polynomial degree of 6. Suggesting that a lower regularization effect was more effective. Interestingly, Lasso's best alpha was 1, indicating regularization was beneficial for Lasso. Removing the paint color from the models actually had a slight detrimental effect, so it would be best to keep this feature. In terms of feature importance, the top features across all models were polynomial representations of age, average miles per year and odometer readings. This indicates that our feature engineering was successful and as expected from our multivariate analysis, mileage and age significantly influence the price. Additionally, our analysis revealed that Ford, Chevy and Toyota were the most popular manufacturers. The univariate analysis showed that vehicles in excellent and good condition were most common with average prices falling in the middle. Vehicle type analysis showed interesting trends. Sedans, SUVs and trucks were the most popular but had distinct average price points. When reporting to the business we will present the Ridge model and talk about the features that were most influential and most popular. We will discus how the most important features affect the price of a vehicle.
</p>

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.